<a href="https://colab.research.google.com/github/tcharos/NLP-Toxicity-Detection/blob/main/AIDL_CS01_NLP_Project_task_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required packages
!pip install langchain-community langchain-text-splitters pypdf sentence-transformers numpy


In [ ]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [ ]:
from unsloth import FastLanguageModel
import torch

### 1. Load and segment a PDF

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
online_document = "https://academicweb.nd.edu/~lemmon/courses/deep-learning/lecture-book/deep-learning-book-2025.pdf"

loader = PyPDFLoader(online_document) # define loader
pages = loader.load() # load

print(f"Loaded {len(pages)} pages")
print(f"First page preview:\n{pages[0].page_content[:500]}")

In [ ]:
# Split into chunks
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=20,
    length_function=len
) # define splitter with chunk size 1000 and 20% overlap

chunks = splitter.split_documents(pages)

print(f"\nCreated {len(chunks)} chunks")
print(f"First chunk:\n{chunks[0].page_content}")
print(f"Metadata: {chunks[0].metadata}")

### 2. Embed and store in numpy array

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np

model_emb = SentenceTransformer('all-MiniLM-L6-v2')

# Extract text and generate embeddings
texts = [doc.page_content for doc in chunks]
embeddings_array = model_emb.encode(texts, convert_to_numpy=True)

# Save and reload example
np.save('embeddings.npy', embeddings_array)
embeddings_array = np.load('embeddings.npy')

### 3. Use an S/LLM from Hugginface for paraphrase

In [ ]:
# use a model, check how to do that in HF, for example check this model card https://huggingface.co/google/gemma-2b
#  or select on from unsloath
# paraphrase 2 times (create the necessary prompt)

In [ ]:
# Load model and tokenizer
model_llm, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct",
    max_seq_length = 2048,
    load_in_4bit = True,
)
FastLanguageModel.for_inference(model_llm)

def get_paraphrases(question, n=2):
    prompt = f"Paraphrase the following question in {n} different ways. Return only the paraphrased questions, one per line:\n\nQuestion: {question}"
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
    outputs = model_llm.generate(**inputs, max_new_tokens=100)
    decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    # split by newline
    return [question] + decoded.split('\n')[-n:]

question = "what is an RNN?"
paraphrased_queries = get_paraphrases(question)

In [ ]:
print("--- Paraphrased Qustion ---")
for i, query in enumerate(paraphrased_queries, 1):
    print(f"Paraphrase {i}: {query}")

### 4. Retrieve 5 most semantically close chunks (cosine similarity) for every paraphrase*, then add threshold 0.3 and select top 3

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

all_indices = []
threshold = 0.3

for q in paraphrased_queries:
    q_emb = model_emb.encode([q])
    sims = cosine_similarity(q_emb, embeddings_array)[0]

    top_5_idx = sims.argsort()[-5:][::-1]
    filtered_idx = [i for i in top_5_idx if sims[i] >= threshold]
    all_indices.extend(filtered_idx)

unique_top_indices = list(dict.fromkeys(all_indices))[:3]
retrieved_context = "\n".join([chunks[i].page_content for i in unique_top_indices])

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

all_queries = [question] + paraphrased_queries
candidate_pool = []
threshold = 0.3

print(f"--- Verification: Searching for {len(all_queries)} query variations ---")

for q in all_queries:
    q_emb = model_emb.encode([q])

    sims = cosine_similarity(q_emb, embeddings_array)[0]

    top_5_idx = sims.argsort()[-5:][::-1]

    for idx in top_5_idx:
        score = sims[idx]
        if score >= threshold:
            candidate_pool.append((idx, score))
            print(f"Match Found! Query: '{q[:30]}...' | Score: {score:.4f} | Index: {idx}")

candidate_pool.sort(key=lambda x: x[1], reverse=True)

final_indices = []
seen_indices = set()

for idx, score in candidate_pool:
    if idx not in seen_indices:
        final_indices.append(idx)
        seen_indices.add(idx)
    if len(final_indices) == 3: # get top 3
        break

print(f"\n--- Final Selection (Top {len(final_indices)} Unique Chunks) ---")
for i, idx in enumerate(final_indices):
    content_preview = chunks[idx].page_content[:150].replace('\n', ' ')
    print(f"{i+1}. [Index {idx}] Content: {content_preview}...")

# Store for 5
retrieved_context = "\n\n".join([chunks[i].page_content for i in final_indices])

In [ ]:
# Re-ranking (not mandatory)

# ----------- Disclaimer - Not mine code (AI generated) but I wanted to try it out -----------

# from sentence_transformers import CrossEncoder

# rerank_model = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

# original_query = all_queries[0]
# pairs = [[original_query, chunk] for chunk in retrieved_context]

# rerank_scores = rerank_model.predict(pairs)

# reranked_results = sorted(
#     zip(retrieved_context, rerank_scores),
#     key=lambda x: x[1],
#     reverse=True
# )

# final_retrieved_context = [chunk for chunk, score in reranked_results[:3]]

# print("Re-ranking complete. Chunks are now ordered by relevance to the original question.")

# ----------- Disclaimer - Not mine code (AI generated) -----------

### 5. Apply the augmentation phase using an SLM (could be the one you have used in step 2 or the only aligned with DPO)

In [ ]:
final_prompt = f"""Use the following context to answer the question.
Context: {retrieved_context}
Question: {paraphrased_queries[0]}
Answer:"""

inputs = tokenizer([final_prompt], return_tensors="pt").to("cuda")
outputs = model_llm.generate(**inputs, max_new_tokens=250)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True)[0])